------
## run 2C_meme_testis_subsample.sh
--------

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoModel
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
from collections import Counter
import re
import ast
import ahocorasick
import os
import difflib
from itertools import combinations
import glob

from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests
from Bio import pairwise2
from Bio import motifs
from Bio import SeqIO
import requests
import json

import json
from pathlib import Path

/home/campus.stonybrook.edu/psurana/anaconda3/envs/dnabert_aug_2025_jupyter/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/campus.stonybrook.edu/psurana/anaconda3/envs/dnabert_aug_2025_jupyter/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
in_path = Path("/data/private/psurana/TSProm/src/files/jobs.json")

with open(in_path, "r") as f:
    jobs = json.load(f)

print(f"Loaded {len(jobs)} jobs from {in_path}")


Loaded 8 jobs from /data/private/psurana/TSProm/src/files/jobs.json


In [4]:
# run it for one job ahead to check
job_to_run = jobs[7]
job_to_run

{'base_model_nm': 'TSp_vs_genNullseqs',
 'subset': 'tspbrain_genNullseqs',
 'lr_dir': 'lr3e-5_ep10',
 'model_path': '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/TSp_vs_genNullseqs/3k/tspbrain_genNullseqs/lr3e-5_ep10/checkpoint-300',
 'data_dir': '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/data/jul_2025/split/TSp_vs_genNullseqs/3k/tspbrain_genNullseqs',
 'res_pdir': '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tspbrain_genNullseqs'}

In [5]:
all_files = glob.glob(job_to_run['res_pdir'] + '/motifs/subsample*/tomtom_out/tomtom.tsv')
all_files

['/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tspbrain_genNullseqs/motifs/subsample_run_1/tomtom_out/tomtom.tsv',
 '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tspbrain_genNullseqs/motifs/subsample_run_3/tomtom_out/tomtom.tsv',
 '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tspbrain_genNullseqs/motifs/subsample_run_4/tomtom_out/tomtom.tsv',
 '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tspbrain_genNullseqs/motifs/subsample_run_6/tomtom_out/tomtom.tsv',
 '/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESU

In [7]:
## for testis only (subsampled results) or liver subsampled results
q_df = pd.concat(
    [pd.read_csv(f, sep='\t', comment='#')
       .assign(subsample=os.path.basename(os.path.dirname(os.path.dirname(f))))
     for f in all_files],
    ignore_index=True
)
filtered_df = q_df[(q_df['p-value'] < 0.01) & (q_df['q-value'] < 0.05) & (q_df['E-value'] < 1.0)]
print(q_df.shape, filtered_df.shape) 
filtered_df.head(2)

(400, 11) (250, 11)


,Query_ID,Target_ID,Optimal_offset,p-value,E-value,q-value,Overlap,Query_consensus,Target_consensus,Orientation,subsample
7,RGYCWVGGGGVGGGG,MA1961.2,-5,3.275400e-08,0.000077,0.000151,10,GGCCTGGGGGCGGGG,GGGGGCGGGGG,+,subsample_run_1
8,RGYCWVGGGGVGGGG,MA1653.2,-5,2.189610e-07,0.000514,0.000505,10,GGCCTGGGGGCGGGG,GGGGGAGGGG,-,subsample_run_1


In [8]:
## for testis only (subsampled results)

# Sort by q-value in ascending order
filtered_df_sorted = filtered_df.sort_values(by='q-value', ascending=True)

df = filtered_df_sorted

# Function to fetch metadata from JASPAR REST API
def fetch_jaspar_metadata(matrix_id):
    url = f"https://jaspar.elixir.no/api/v1/matrix/{matrix_id}"
    try:
        res = requests.get(url, headers={"Accept": "application/json"}, timeout=10)
        if res.status_code == 200:
            info = res.json()
            return {
                "TF_name": info.get("name"),
                "Class": info.get("class"),
                "Family": info.get("family"),
                "Tax_group": info.get("tax_group"),
                "Species": info.get("species")
            }
        else:
            return {"TF_name": None, "Class": None, "Family": None,
                    "Tax_group": None, "Species": None}
    except Exception as e:
        print(f"⚠️ Error fetching {matrix_id}: {e}")
        return {"TF_name": None, "Class": None, "Family": None,
                "Tax_group": None, "Species": None}

# Apply to all unique Target_IDs
meta_dict = {}
for tid in df["Target_ID"].unique():
    meta_dict[tid] = fetch_jaspar_metadata(tid)

# Add metadata to dataframe
meta_df = pd.DataFrame.from_dict(meta_dict, orient="index").reset_index().rename(columns={"index": "Target_ID"})
df_annot = df.merge(meta_df, on="Target_ID", how="left")

df_annot = df_annot[df_annot['Tax_group'] == "vertebrates"]
print(df_annot.shape)


(210, 16)


In [9]:
job_to_run['res_pdir']

'/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tspbrain_genNullseqs'

In [44]:

df_annot.to_csv(job_to_run['res_pdir'] + '/motifs/subsample_all' + "/filtered_tomtom_results.tsv", sep='\t', index=False)


# biclustering
# make binary matrix and try

In [132]:
from pathlib import Path
import pandas as pd

base = Path("/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10")

# search recursively for all prePWM and tomtom files
prepwms = list(base.rglob("4sig_motifs_prePWM.csv"))
tomtoms = list(base.rglob("filtered_tomtom_results.tsv"))

In [134]:
# Find all relevant files
prepwms = [p for p in base.rglob("4sig_motifs_prePWM.csv") if "tsptestis" in str(p)]
tomtoms = [p for p in base.rglob("filtered_tomtom_results.tsv") if "tsptestis" in str(p)]

for p in prepwms: print(p)

for p in tomtoms: print(p)


/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tsptestis_genNullseqs/4sig_motifs_prePWM.csv
/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_nonProm_3k_tsptestis_nonPromHu/4sig_motifs_prePWM.csv
/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tsptestis_genNullseqs/motifs/subsample_all/filtered_tomtom_results.tsv
/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tsptestis_genNullseqs/motifs/tomtom_out_label_1/filtered_tomtom_results.tsv
/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_nonProm_3k_tsptestis_nonPromH

In [135]:
def short_name(p: Path) -> str:
    return p.parents[2].name if "motifs" in p.parts else p.parent.name


# Read files with those names as keys
dfs_prePWM = {short_name(p): pd.read_csv(p) for p in prepwms}

p = "/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tsptestis_genNullseqs/motifs/subsample_all/filtered_tomtom_results.tsv"
dfs_tomtom = pd.read_csv(p, sep="\t")

In [136]:
dfs_tomtom.keys()

Index(['Query_ID', 'Target_ID', 'Optimal_offset', 'p-value', 'E-value',
       'q-value', 'Overlap', 'Query_consensus', 'Target_consensus',
       'Orientation', 'subsample', 'TF_name', 'Class', 'Family', 'Tax_group',
       'Species'],
      dtype='object')

In [14]:
# print(dfs_prePWM['TSp_vs_genNullseqs_3k_tspbrain_genNullseqs'].columns)
# dfs_prePWM['TSp_vs_genNullseqs_3k_tspbrain_genNullseqs'].head(2)

In [137]:
print(dfs_tomtom.head(2))
print(dfs_tomtom.shape)
# for testis read subsample_all results
print(dfs_tomtom.Query_consensus.value_counts())

               Query_ID Target_ID  Optimal_offset       p-value   E-value  \
0  SCCRGSDGBBBRRSMVRGDG  MA1929.2               8  1.723770e-09  0.000004   
1  GSCDKGVSVMNVNGSNGRGR  MA1929.2               4  1.555120e-08  0.000036   

    q-value  Overlap       Query_consensus                 Target_consensus  \
0  0.000008       20  CCCAGCAGGGCAGGACAGAG  CGCCCCCTGGTGGCCACAGCTGGAACTGCAG   
1  0.000036       20  GCCGGGGGAAAATGGGGGGG  CGCCCCCTGGTGGCCACAGCTGGAACTGCAG   

  Orientation         subsample TF_name                         Class  \
0           -   subsample_run_7    CTCF  ['C2H2 zinc finger factors']   
1           -  subsample_run_10    CTCF  ['C2H2 zinc finger factors']   

                                  Family    Tax_group  \
0  ['More than 3 adjacent zinc fingers']  vertebrates   
1  ['More than 3 adjacent zinc fingers']  vertebrates   

                                      Species  
0  [{'tax_id': 9606, 'name': 'Homo sapiens'}]  
1  [{'tax_id': 9606, 'name': 'Homo sapiens

In [138]:
tiss_data = pd.read_csv('/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/data/jul_2025/TSp_vs_genNullseqs/3k/tsptestis_2kTSS1k_rep_seq_balanced.csv')
tiss_data1 = tiss_data[tiss_data['Label']==1]
tiss_data0 = tiss_data[tiss_data['Label']==0]
print(tiss_data1.shape, tiss_data0.shape)

(9559, 3) (9559, 3)


In [139]:
motif_df = dfs_tomtom
motifs_all = (
    motif_df['Query_consensus']
    .dropna()
    .astype(str)
    .str.strip()
    .str.upper()
    .unique()
    .tolist()
)

# Group motifs by length for a small speedup
from collections import defaultdict
motifs_by_len = defaultdict(list)
for m in motifs_all:
    motifs_by_len[len(m)].append(m)


In [140]:
THRESH = 0.80

def best_match_in_seq(seq: str):
    """
    Return dict with best match info if identity >= THRESH, else None.
    Checks all motif lengths and all positions.
    """
    if not isinstance(seq, str):
        return None
    s = seq.upper()
    n = len(s)
    best = None  # {'motif','pos','window','identity'}

    for mlen, motif_list in motifs_by_len.items():
        if mlen == 0 or mlen > n:
            continue

        # Slide window of size mlen across the sequence
        for i in range(n - mlen + 1):
            window = s[i:i+mlen]
            # Compare this window to each motif of this length
            # (Simple exact-char comparison; ambiguous bases count as mismatch)
            for mot in motif_list:
                matches = sum(ch1 == ch2 for ch1, ch2 in zip(window, mot))
                ident = matches / mlen
                if best is None or ident > best['identity']:
                    best = {'motif': mot, 'pos': i, 'window': window, 'identity': ident}

    if best and best['identity'] >= THRESH:
        return best
    return None

def annotate_row(seq: str):
    hit = best_match_in_seq(seq)
    if hit is None:
        return pd.Series({
            'motif_found': pd.NA,
            'motif_pos': pd.NA,
            'matched_subseq': pd.NA,
            'match_identity': 0.0,
            'Status': 0
        })
    else:
        return pd.Series({
            'motif_found': hit['motif'],
            'motif_pos': hit['pos'],
            'matched_subseq': hit['window'],
            'match_identity': round(hit['identity'], 4),
            'Status': 1
        })

In [141]:
# --- 2) Apply to your tiss_data DataFrame (must have a 'Sequence' column) ---
tiss_data1 = tiss_data1.copy()
tiss_data1[['motif_found','motif_pos','matched_subseq','match_identity','Status']] = (
    tiss_data1['Sequence'].apply(annotate_row)
)

# --- 2) Apply to your tiss_data DataFrame (must have a 'Sequence' column) ---
tiss_data0 = tiss_data0.copy()
tiss_data0[['motif_found','motif_pos','matched_subseq','match_identity','Status']] = (
    tiss_data0['Sequence'].apply(annotate_row)
)

In [142]:
print(tiss_data1.head(2))
print(tiss_data1.shape)
print(tiss_data0.shape)

                                                 Header  \
9559  chr5_90408011_90411011_90410011_+_testis_2kTSS...   
9560   chr20_5530878_5533878_5532878_+_testis_2kTSS1k_1   

                                               Sequence  Label  \
9559  AACTTCGCAAAACTAAAGAGTGTTACAGATTATTAAAAGAAATGAG...      1   
9560  TAACATAATGAGACCGAGCTTCACCATGTTGTCTAGACTGTTCTCA...      1   

               motif_found motif_pos        matched_subseq  match_identity  \
9559  CTCAAATAAAAAAAAAAAAA       786  CTCCAAAAAAAAAAAAAAAA          0.9000   
9560       ACTAAAAATAAAAAA      1579       ACTAAAAATACAAAA          0.9333   

      Status  
9559       1  
9560       1  
(9559, 8)
(9559, 8)


In [143]:
print(tiss_data1['Status'].value_counts())
print(tiss_data1['match_identity'].value_counts())

print(tiss_data0['match_identity'].value_counts())

Status
1    8994
0     565
Name: count, dtype: int64
match_identity
1.0000    3534
0.8000    1187
0.9333    1090
0.8667     941
0.9286     880
0.8571     762
0.0000     565
0.9500     217
0.8500     100
0.9000      79
0.8462      66
0.8125      43
0.9231      28
0.8889      26
0.8750      26
0.8333       8
0.9444       5
0.9375       2
Name: count, dtype: int64
match_identity
1.0000    3577
0.8000    1236
0.9333     964
0.0000     898
0.8667     797
0.8571     769
0.9286     748
0.9500     190
0.8462      86
0.8500      82
0.9000      70
0.8125      48
0.9231      36
0.8889      26
0.8750      21
0.8333       7
0.9444       3
0.9375       1
Name: count, dtype: int64


In [144]:
import numpy as np
tiss_data0['Status_motif'] = np.where(tiss_data0['match_identity'] >= 0.85, 1, 0)
print(tiss_data0['Status_motif'].value_counts())

tiss_data1['Status_motif'] = np.where(tiss_data1['match_identity'] >= 0.85, 1, 0)
print(tiss_data1['Status_motif'].value_counts())

Status_motif
1    7284
0    2275
Name: count, dtype: int64
Status_motif
1    7690
0    1869
Name: count, dtype: int64


In [145]:
tiss_data1.head(2)

,Header,Sequence,Label,motif_found,motif_pos,matched_subseq,match_identity,Status,Status_motif
9559,chr5_90408011_90411011_90410011_+_testis_2kTSS...,AACTTCGCAAAACTAAAGAGTGTTACAGATTATTAAAAGAAATGAG...,1,CTCAAATAAAAAAAAAAAAA,786,CTCCAAAAAAAAAAAAAAAA,0.9000,1,1
9560,chr20_5530878_5533878_5532878_+_testis_2kTSS1k_1,TAACATAATGAGACCGAGCTTCACCATGTTGTCTAGACTGTTCTCA...,1,ACTAAAAATAAAAAA,1579,ACTAAAAATACAAAA,0.9333,1,1


In [146]:
tiss_data0['motif_found'].nunique()

20

In [147]:
# construct a binary matrix

df = tiss_data1
mat = (
    df.pivot_table(
        index="Header",
        columns="motif_found",
        values="Status_motif",
        aggfunc="max",        # if duplicates exist, keep 1 if any match
        fill_value=0
    )
    .astype("int8")
)
mat.shape

# Remove all-zero columns
mat = mat.loc[:, (mat != 0).any(axis=0)]

# Remove all-zero rows
mat = mat.loc[~(mat == 0).all(axis=1)]

print(mat.shape)
mat.head(3)

(7690, 19)


motif_found,ACTAAAAATAAAAAA,CCCAGCAGGGCAGGACAGAG,CCGGCCTCGGCCTCCTCCGC,CCTCAGTCGCAGCCCCGTCC,CCTGCCTGGCCCGC,CGAGGTCAGGAGT,CTCAAATAAAAAAAAAAAAA,GCCCCCGCCTCCCGC,GCCGGGGGAAAATGGGGGGG,GCCTGTAATCCCAG,GGCGGCGGCGGCGGG,GGCGGGGGGGGCGGGTGGGG,GGGAGGATCTCTTTAGCC,GGGAGGCTAAGGTAAG,GTGGCCAGGGTAAGG,TCTTTAATAAAAAAAAAAAA,TTTTTTTATTTATTATATTT,TTTTTTTATTTTTAG,TTTTTTTTTTTTTT
Header,,,,,,,,,,,,,,,,,,,
chr10_100227667_100230667_100229667_+_testis_2kTSS1k_1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
chr10_100328940_100331940_100329940_-_testis_2kTSS1k_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
chr10_100345390_100348390_100346390_-_testis_2kTSS1k_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [148]:
import numpy as np
import pandas as pd
from sklearn.cluster import SpectralCoclustering
from sklearn.metrics import silhouette_score

# --- 1️⃣ Clean your binary matrix ---
M = (
    mat.replace([np.inf, -np.inf], np.nan)
       .fillna(0)
       .astype(int)
)
M = M.loc[M.sum(axis=1) > 0, M.sum(axis=0) > 0]

# --- 2️⃣ Try several cluster numbers and pick the best ---
scores = {}
best_k, best_score = None, -1
for k in range(2, 10):
    model = SpectralCoclustering(n_clusters=k, random_state=0)
    model.fit(M)
    if len(set(model.row_labels_)) > 1:
        sc = silhouette_score(M, model.row_labels_, metric="cosine")
        scores[k] = sc
        if sc > best_score:
            best_k, best_score = k, sc

print("Silhouette scores:", scores)
print(f"✅ Best k = {best_k} (score = {best_score:.3f})")

# --- 3️⃣ Fit final model ---
co = SpectralCoclustering(n_clusters=best_k, random_state=0)
co.fit(M)

# --- 4️⃣ Make a simple summary table ---
summary = []
for k in range(best_k):
    r_idx = np.where(co.row_labels_ == k)[0]
    c_idx = np.where(co.column_labels_ == k)[0]
    summary.append({
        "Cluster": k,
        "Num_Headers": len(r_idx),
        "Num_Motifs": len(c_idx),
        "Headers_Sample": ", ".join(M.index[r_idx][:5]) + ("..." if len(r_idx) > 5 else ""),
        "Motifs_Sample": ", ".join(M.columns[c_idx][:5]) + ("..." if len(c_idx) > 5 else "")
    })

summary_df = pd.DataFrame(summary)
print("\n=== Bicluster Summary ===")

Silhouette scores: {2: 0.3850555868829795, 3: 0.2564220973923718, 4: 0.5035235641137564, 5: 0.4010137406295454, 6: 0.4182005068638442, 7: 0.469860461976331, 8: 0.47121903473113297, 9: 0.8033114861144386}
✅ Best k = 9 (score = 0.803)

=== Bicluster Summary ===


In [149]:
summary_df["Cluster_Type"] = np.where(summary_df["Num_Headers"] >= 10, "Major", "Minor")

In [150]:
summary_df['Headers_Sample'][1]

'chr10_116847499_116850499_116849499_+_testis_2kTSS1k_1, chr10_132349607_132352607_132351607_+_testis_2kTSS1k_1, chr10_4837784_4840784_4839784_+_testis_2kTSS1k_1, chr11_12274080_12277080_12276080_+_testis_2kTSS1k_1, chr11_2914948_2917948_2916948_+_testis_2kTSS1k_1...'

In [151]:
motif_df.Query_consensus.value_counts()

Query_consensus
GGCGGCGGCGGCGGG         14
CCGGCCTCGGCCTCCTCCGC    11
GCCCCCGCCTCCCGC          9
CCTCAGTCGCAGCCCCGTCC     9
GGCGGGGGGGGCGGGTGGGG     8
GCCGGGGGAAAATGGGGGGG     6
CCCAGCAGGGCAGGACAGAG     5
GTGGCCAGGGTAAGG          4
GGGAGGATCTCTTTAGCC       2
CCTGCCTGGCCCGC           2
TCCGGGTCTTTAGCCGCCTC     2
TTTTTTTATTTATTATATTT     2
CGAGGTCAGGAGT            2
TTTTTTTATTTTTAG          2
GGGAGGCTAAGGTAAG         1
CTCAAATAAAAAAAAAAAAA     1
TCTTTAATAAAAAAAAAAAA     1
TTTTTTTTTTTTTT           1
ACTAAAAATAAAAAA          1
GCCTGTAATCCCAG           1
Name: count, dtype: int64

In [152]:
motif_df.head(2)

,Query_ID,Target_ID,Optimal_offset,p-value,E-value,q-value,Overlap,Query_consensus,Target_consensus,Orientation,subsample,TF_name,Class,Family,Tax_group,Species
0,SCCRGSDGBBBRRSMVRGDG,MA1929.2,8,1.723770e-09,0.000004,0.000008,20,CCCAGCAGGGCAGGACAGAG,CGCCCCCTGGTGGCCACAGCTGGAACTGCAG,-,subsample_run_7,CTCF,['C2H2 zinc finger factors'],['More than 3 adjacent zinc fingers'],vertebrates,"[{'tax_id': 9606, 'name': 'Homo sapiens'}]"
1,GSCDKGVSVMNVNGSNGRGR,MA1929.2,4,1.555120e-08,0.000036,0.000036,20,GCCGGGGGAAAATGGGGGGG,CGCCCCCTGGTGGCCACAGCTGGAACTGCAG,-,subsample_run_10,CTCF,['C2H2 zinc finger factors'],['More than 3 adjacent zinc fingers'],vertebrates,"[{'tax_id': 9606, 'name': 'Homo sapiens'}]"


In [153]:
# 1) Normalize and aggregate motif info (dedupe by Target_consensus)
m = motif_df.dropna(subset=["Query_consensus"]).copy()
m["key"] = m["Query_consensus"].str.strip().str.upper()

agg = (m.groupby("key", as_index=True)
        .agg({
            "TF_name":  lambda s: sorted({str(x) for x in s.dropna()}),
            "Class":    lambda s: sorted({str(x) for x in s.dropna()}),
            "Target_ID":lambda s: sorted({str(x) for x in s.dropna()}),
        }))

# 2) Build a lookup dict (key -> dict of lists)
motif_info = agg.to_dict(orient="index")

def get_tf_info(motif_str: str):
    if pd.isna(motif_str) or not str(motif_str).strip():
        return pd.Series({"TF_names": None, "TF_classes": None, "Target_IDs": None})
    toks = [t.strip() for t in str(motif_str).split(",") if t.strip() and t.strip() != "..."]

    tfs, classes, ids = set(), set(), set()
    for t in toks:
        info = motif_info.get(t.upper())
        if info:
            tfs.update(info["TF_name"])
            classes.update(info["Class"])
            ids.update(info["Target_ID"])

    return pd.Series({
        "TF_names":   ", ".join(sorted(tfs)) if tfs else None,
        "TF_classes": ", ".join(sorted(classes)) if classes else None,
        "Target_IDs": ", ".join(sorted(ids)) if ids else None
    })

# 3) Apply to summary_df["Motifs_Sample"]
summary_df[["TF_names", "TF_classes", "Target_IDs"]] = summary_df["Motifs_Sample"].apply(get_tf_info)

pth = "/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tsptestis_genNullseqs"
summary_df.to_csv(pth + '/bicluster/bicluster_res_query_consesnsus_Label1.csv')
summary_df

,Cluster,Num_Headers,Num_Motifs,Headers_Sample,Motifs_Sample,Cluster_Type,TF_names,TF_classes,Target_IDs
0,0,2971,2,chr10_100328940_100331940_100329940_-_testis_2...,"CGAGGTCAGGAGT, TTTTTTTTTTTTTT",Major,"Nr1H2, Nr1H4, ZNF384","['C2H2 zinc finger factors'], ['Nuclear recept...","MA1110.3, MA1125.2, MA1996.2"
1,1,149,2,chr10_116847499_116850499_116849499_+_testis_2...,"CCTGCCTGGCCCGC, GTGGCCAGGGTAAGG",Major,"PRDM9, RFX1, RFX2, RFX3, RFX5, ZNF213","['C2H2 zinc finger factors'], ['Fork head/wing...","MA0509.3, MA0510.3, MA0600.3, MA0798.3, MA1723..."
2,2,953,2,chr10_101235269_101238269_101237269_+_testis_2...,"ACTAAAAATAAAAAA, CTCAAATAAAAAAAAAAAAA",Major,"Hmga1, ZNF384","['C2H2 zinc finger factors'], ['TATA-binding p...","MA1125.2, MA2124.1"
3,3,1377,5,chr10_102916317_102919317_102918317_+_testis_2...,"CCCAGCAGGGCAGGACAGAG, GGCGGCGGCGGCGGG, GGGAGGA...",Major,"CTCF, FOXD3, GATA1::TAL1, KLF12, KLF16, MEF2A,...","['C2H2 zinc finger factors'], ['Fork head/wing...","MA0041.3, MA0052.5, MA0079.5, MA0140.3, MA0145..."
4,4,56,2,chr10_10945602_10948602_10946602_-_testis_2kTS...,"CCTCAGTCGCAGCCCCGTCC, TCTTTAATAAAAAAAAAAAA",Major,"CTCF, NHLH2, PATZ1, PRDM9, SP2, ZNF213, ZNF354...","['Basic helix-loop-helix factors (bHLH)'], ['C...","MA0516.3, MA1529.2, MA1714.2, MA1721.2, MA1723..."
5,5,29,1,chr11_64283257_64286257_64285257_+_testis_2kTS...,GGCGGGGGGGGCGGGTGGGG,Major,"CTCF, KLF4, PRDM9, TFAP2A, ZNF320, ZNF454, ZNF93","['Basic helix-span-helix factors (bHSH)'], ['C...","MA0039.5, MA0810.2, MA1712.2, MA1721.2, MA1723..."
6,6,3,1,chr10_49677651_49680651_49679651_+_testis_2kTS...,GCCGGGGGAAAATGGGGGGG,Minor,"CTCF, NFIB, PRDM9, ZFP14, ZNF816","['C2H2 zinc finger factors'], ['SMAD/NF-1 DNA-...","MA1643.2, MA1719.2, MA1723.2, MA1929.2, MA1930..."
7,7,2,1,chr2_131443587_131446587_131444587_-_testis_2k...,CCGGCCTCGGCCTCCTCCGC,Minor,"CTCF, PATZ1, PRDM9, ZFP14, ZNF135, ZNF320, ZNF...",['C2H2 zinc finger factors'],"MA0146.3, MA1587.1, MA1596.1, MA1712.2, MA1714..."
8,8,2150,3,chr10_100227667_100230667_100229667_+_testis_2...,"GCCCCCGCCTCCCGC, GCCTGTAATCCCAG, GGGAGGCTAAGGTAAG",Major,"CTCF, Crx, PATZ1, PRDM9, SP1, SP2, ZFP14, ZNF3...","['C2H2 zinc finger factors'], ['Homeo domain f...","MA0079.5, MA0467.3, MA0516.3, MA1596.1, MA1712..."


## go to http://www.zpliulab.cn/RegNetwork/search
## then get TF-TF network

In [127]:
df_regtwork_bhlh = pd.read_csv(pth + '/bicluster/bhlh_brain.txt', sep="\t")
df_regtwork_znfinger = pd.read_csv(pth + '/bicluster/bshs_znFinger_brain.txt', sep="\t")

df_regtwork_bhlh.head(2)
df_regtwork_znfinger.head(2)

,Regulator Symbol,Regulator ID,Target Symbol,Target ID,Regulator Type,Target Type,Confidence,Score,References
0,SP1,NCBI:6667,RARA,NCBI:5914,TF,TF,High,0.999789,9626663(Experimental Research)
1,TFAP2A,NCBI:7020,NR3C1,NCBI:2908,TF,TF,High,0.997928,7794935(Experimental Research)


In [129]:
bhlh = ["BHLHE40", "EPAS1", "HES6", "HES7", "HEY2", "HIF1A", "MYC", "SOHLH2"]
znfinger =[
    "KLF1", "KLF10", "KLF12", "KLF14", "KLF15", "KLF16",
    "KLF4", "KLF5", "KLF7", "MAZ", "PATZ1", "SP1", "SP2",
    "SP4", "SP8", "SP9", "TFAP2A", "ZNF148", "ZNF281",
    "ZNF320", "ZNF740"
]

df_bhlh_network = df_regtwork_bhlh[
    (df_regtwork_bhlh['Regulator Symbol'].isin(bhlh)) &
    (df_regtwork_bhlh['Target Symbol'].isin(bhlh))
]

df_znfinger_network = df_regtwork_znfinger[
    (df_regtwork_znfinger['Regulator Symbol'].isin(znfinger)) &
    (df_regtwork_znfinger['Target Symbol'].isin(znfinger))
]

In [130]:
df_bhlh_network

,Regulator Symbol,Regulator ID,Target Symbol,Target ID,Regulator Type,Target Type,Confidence,Score,References
89,HIF1A,NCBI:3091,BHLHE40,NCBI:8553,TF,TF,High,0.979643,NaN
98,MYC,NCBI:4609,BHLHE40,NCBI:8553,TF,TF,High,0.949108,NaN
225,BHLHE40,NCBI:8553,HIF1A,NCBI:3091,TF,TF,High,0.949108,NaN
267,MYC,NCBI:4609,HIF1A,NCBI:3091,TF,TF,High,0.979643,NaN
296,BHLHE40,NCBI:8553,BHLHE40,NCBI:8553,TF,TF,High,0.979643,NaN
606,HEY2,NCBI:23493,MYC,NCBI:4609,TF,TF,High,0.800000,NaN
640,MYC,NCBI:4609,HES6,NCBI:55502,TF,TF,High,0.899112,NaN
671,HIF1A,NCBI:3091,HIF1A,NCBI:3091,TF,TF,High,0.800000,14744852(Experimental Research)
832,MYC,NCBI:4609,MYC,NCBI:4609,TF,TF,High,0.959645,NaN
972,HEY2,NCBI:23493,HEY2,NCBI:23493,TF,TF,High,0.959645,NaN


In [131]:
df_znfinger_network

,Regulator Symbol,Regulator ID,Target Symbol,Target ID,Regulator Type,Target Type,Confidence,Score,References
19,SP1,NCBI:6667,SP1,NCBI:6667,TF,TF,High,0.994820,NaN
84,SP1,NCBI:6667,KLF10,NCBI:7071,TF,TF,High,0.989731,NaN
166,SP1,NCBI:6667,MAZ,NCBI:4150,TF,TF,High,0.949108,NaN
179,SP1,NCBI:6667,KLF7,NCBI:8609,TF,TF,High,0.949108,NaN
220,TFAP2A,NCBI:7020,MAZ,NCBI:4150,TF,TF,High,0.949108,NaN
324,SP1,NCBI:6667,KLF5,NCBI:688,TF,TF,High,0.949108,NaN
344,TFAP2A,NCBI:7020,TFAP2A,NCBI:7020,TF,TF,High,0.979643,NaN
463,TFAP2A,NCBI:7020,SP1,NCBI:6667,TF,TF,High,0.979643,NaN
486,TFAP2A,NCBI:7020,KLF12,NCBI:11278,TF,TF,High,0.979643,10704285(Experimental Research)
530,SP1,NCBI:6667,SP4,NCBI:6671,TF,TF,High,0.979643,NaN


In [172]:
# testis data
pth = "/data/projects/dna/pallavi/DNABERT_runs/DATA_RUN/dnabert2_FineTune_Zhihan_attention_extracted/july_2025_mmseq/RESULT/lr3e-5_ep10/TSp_vs_genNullseqs_3k_tsptestis_genNullseqs"

dfa = pd.read_csv(pth + '/bicluster/clust0_testis.txt', sep="\t")
dfb = pd.read_csv(pth + '/bicluster/clust1_testis.txt', sep="\t")
dfc = pd.read_csv(pth + '/bicluster/clust3_testis.txt', sep="\t")
dfd = pd.read_csv(pth + '/bicluster/clust_4_testis.txt', sep="\t")
dfe = pd.read_csv(pth + '/bicluster/clust5_testis.txt', sep="\t")


In [173]:
# === Cluster / Module 1 ===
tf_list_1 = ["Nr1H2", "Nr1H4", "ZNF384"]

# === Cluster / Module 2 ===
tf_list_2 = ["PRDM9", "RFX1", "RFX2", "RFX3", "RFX5", "ZNF213"]

# === Cluster / Module 3 ===
tf_list_3 = [
    "CTCF", "FOXD3", "GATA1::TAL1", "KLF12", "KLF16", "MEF2A", "MEF2C",
    "PATZ1", "PRDM9", "SP1", "SP4", "SP5", "Tfcp2l1", "Wt1", "ZFP14",
    "ZNF148", "ZNF263", "ZNF281", "ZNF320", "ZNF354A", "ZNF460"
]

# === Cluster / Module 4 ===
tf_list_4 = [
    "CTCF", "Crx", "PATZ1", "PRDM9", "SP1", "SP2",
    "ZFP14", "ZNF320", "ZNF454", "ZNF460", "ZNF93"
]

tf_list_5 = [
    "CTCF", "NHLH2", "PATZ1", "PRDM9",
    "SP2", "ZNF213", "ZNF354A", "ZNF675", "ZNF93"
]



df1=dfa[
    (dfa['Regulator Symbol'].isin(tf_list_1)) &
    (dfa['Target Symbol'].isin(tf_list_1)) &
    (dfa['Regulator Symbol'] != dfa['Target Symbol'])
]

df2=dfb[
    (dfb['Regulator Symbol'].isin(tf_list_2)) &
    (dfb['Target Symbol'].isin(tf_list_2)) &
    (dfb['Regulator Symbol'] != dfb['Target Symbol'])
]

df3=dfc[
    (dfc['Regulator Symbol'].isin(tf_list_3)) &
    (dfc['Target Symbol'].isin(tf_list_3))&
    (dfc['Regulator Symbol'] != dfc['Target Symbol'])
]

df4=dfd[
    (dfd['Regulator Symbol'].isin(tf_list_4)) &
    (dfd['Target Symbol'].isin(tf_list_4)) &
    (dfd['Regulator Symbol'] != dfd['Target Symbol'])
]

df5=dfe[
    (dfe['Regulator Symbol'].isin(tf_list_5)) &
    (dfe['Target Symbol'].isin(tf_list_5)) &
    (dfe['Regulator Symbol'] != dfe['Target Symbol'])
]

In [180]:
print(df2.shape, df3.shape)

(4, 9) (10, 9)


In [181]:
import openpyxl
with pd.ExcelWriter(pth + '/bicluster/clust_all_RegNetwork' ".xlsx") as writer:
    df2.to_excel(writer, sheet_name="clust1", index=False)
    df3.to_excel(writer, sheet_name="clust3", index=False)
